In [9]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

from datetime import date, datetime, timedelta
import time

t0 = time.time()

# Read recipe inputs
NAFCUSTOMER_REVENUE_AGGREGATED = dataiku.Dataset("NAFCUSTOMER_REVENUE_AGGREGATED")
NAFCUSTOMER_REVENUE_AGGREGATED_df = NAFCUSTOMER_REVENUE_AGGREGATED.get_dataframe()

t1 = time.time()
print("load duration", (t1-t0)/60.0, "minutes")

print(len(NAFCUSTOMER_REVENUE_AGGREGATED_df))
NAFCUSTOMER_REVENUE_AGGREGATED_df.head()

load duration 5.2844768563906355 minutes
18075044


,CUSTOMER_ID,CUSTOMER,REVENUE_DATE,REVENUE_YEAR,SETUP_DATE,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY,TOTAL_ACTIVE_CARD_COUNT
0,77514713.0,SAFELITE GROUP INC,2019,10,2010-09-03 07:00:00+00:00,51000.0,10100.0,3100.0,22630.68,2311614.46,863540.6420,6784.0
1,77420042.0,NEW COVENANT CHRISTIAN CENTER INTERNATIONAL INC,2019,6,2018-12-03 08:00:00+00:00,11200.0,15100.0,4000.0,0.00,254.08,80.3080,2.0
2,77131455.0,CHRISTOPHER DUMENKO,2021,5,2011-02-08 08:00:00+00:00,20800.0,15100.0,4000.0,4.94,522.66,127.6998,2.0
3,77088145.0,BLOOMSBURG VOLUNTEER AMBULANCE ASSOCIATION INC,2019,5,2012-09-13 07:00:00+00:00,11800.0,15100.0,4120.0,56.40,2394.82,775.0990,3.0
4,77283958.0,HOMESTEAD DESIGN LLC,2020,9,2013-05-17 07:00:00+00:00,11900.0,15100.0,4120.0,0.78,28.00,12.9680,1.0


In [10]:
print(len(NAFCUSTOMER_REVENUE_AGGREGATED_df))
df = NAFCUSTOMER_REVENUE_AGGREGATED_df

18075044


In [11]:
df.head()

,CUSTOMER_ID,CUSTOMER,REVENUE_DATE,REVENUE_YEAR,SETUP_DATE,BI_CUS_PAR,BI_PRODUCT,BI_MRU,REVENUE_AMOUNT_USD,GROSS_SPEND_AMOUNT,PURCHASE_GALLONS_QTY,TOTAL_ACTIVE_CARD_COUNT
0,77514713.0,SAFELITE GROUP INC,2019,10,2010-09-03 07:00:00+00:00,51000.0,10100.0,3100.0,22630.68,2311614.46,863540.6420,6784.0
1,77420042.0,NEW COVENANT CHRISTIAN CENTER INTERNATIONAL INC,2019,6,2018-12-03 08:00:00+00:00,11200.0,15100.0,4000.0,0.00,254.08,80.3080,2.0
2,77131455.0,CHRISTOPHER DUMENKO,2021,5,2011-02-08 08:00:00+00:00,20800.0,15100.0,4000.0,4.94,522.66,127.6998,2.0
3,77088145.0,BLOOMSBURG VOLUNTEER AMBULANCE ASSOCIATION INC,2019,5,2012-09-13 07:00:00+00:00,11800.0,15100.0,4120.0,56.40,2394.82,775.0990,3.0
4,77283958.0,HOMESTEAD DESIGN LLC,2020,9,2013-05-17 07:00:00+00:00,11900.0,15100.0,4120.0,0.78,28.00,12.9680,1.0


In [0]:
df['REVENUE_DATE']

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#TENURE_CALC_df = NAFCUSTOMER_REVENUE_AGGREGATED_df # For this sample code, simply copy input to output


# Write recipe outputs
#TENURE_CALC = dataiku.Dataset("TENURE_CALC")
#TENURE_CALC.write_with_schema(TENURE_CALC_df)